# Part 1.3: Ribo-seq quality control and downstream analysis of the Rp-Bp results

Through these sections, you wil learn how to use `matplotlib` to visualise the results.

## Sections:
   - 1.3.1 Ribo-seq quality control how-to.
   - 1.3.2 Preprocessing analysis: standard rpbp preprocessing and more.
   - 1.3.3 Prediction analysis: understand the output of rpbp.

## Questions & Objectives:
   - Learn how to identify "good quality" Ribo-seq data.
   - Learn how to visualise the results.
   - Understand and use the output of rpbp (ORF predictions).

### After I will be able to:
   - run the rpbp downstream analysis pipeline and assess the quality of the data;
   - use the ORF predictions for follow-up studies.
   

## 1.3.1 Ribo-seq quality control how-to

Depending on the efficiency of the rRNA removal step of the experimental protocol, small structured RNAs (rRNAs, tRNAs, or snoRNAs) may have to be removed in a first pre-processing step. The remaining reads are mapped using s splice-aware aligner (`STAR`), but reads can still map to multiple locations. Different strategies can be implemented to either rescue the reads, keep one primary alignment per read, or discard all multi-mapping reads altogether. Finally, only periodic reads are kept for the analysis. We will use the data from the output of `Flexbar` (reads filtered for quality, trimmed from adapters), `Bowtie2` (reads mapping to rRNA, clean reads), `STAR` (unique reads) and `rpbp` (periodic reads) to quantify the amount of reads filtered out at each step of the pipeline.

In a high-quality Ribo-seq library, reads mostly map to coding sequences (CDS or Canonical) (typically >85%) and to the 5'UTR (up to 10%). A smaller proportion map to the 3'UTR. The amount of reads mapping to non-coding regions can vary, but in general the signal is not very strong. Using the results of the pipeline, we will explore how many ORFs are predicted in each regions. It is also possible to use other tools such as `bedtools coverage`, that uses the mapped data (before translation prediction).

A characteristic feature of a high-quality Ribo-seq library is its read-length distribution, which typically peaks around 29 nt in eukaryotic organisms, however broader distributions can be observed under different protocols, depending on the nuclease treatment, the drugs/inhibitors used, *etc.* It is also known that different ribosomal conformation correspond to distinct read-length distributions, and that these can also be affected by ribosomes belonging to different pools (mitochondrial ribosomes were shown previously to display a bimodal distribution, compared to cytosolic-derived fragments). All these considerations must be taken into account when analysing the distribution of read lengths.

We will briefly explore these and other aspects graphically, using the data from the example. We will be using the full data from 4 biological replicates. This data has been prepared before the course. 

&#9655;  **Self-guided learning** As a self-guided learning exercise, you can try to use this notebook and perform the analysis of the downsampled data that we ran in the previous notebook. Note that you will first need to run the standard `rpbp` preprocessing analysis (see further below). With the output of this analysis, you will be able to run the first 6 code cells below (*read filtering count*).



## 1.3.2 Preprocessing analysis: standard rpbp preprocessing and more


***
<font color=red>**Note** The cells below contain "code", so we will need to run them one after the other.</font> 

In [60]:
import pandas as pd
import numpy as np
import os
import yaml

from collections import defaultdict

from argparse import Namespace

import pbio.misc.logging_utils as logging_utils
import pbio.misc.mpl_utils as mpl_utils
import pbio.ribo.ribo_utils as ribo_utils

args = Namespace()
logger = logging_utils.get_ipython_logger()

In [64]:
# graphics

%load_ext autoreload
%autoreload 2
%matplotlib inline

import matplotlib
import matplotlib.ticker as mtick
from matplotlib.font_manager import FontProperties
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator

import seaborn as sns
sns.set({"ytick.direction": u'out'}, style='white') #color_codes=True, palette='muted')

params = {
   'axes.labelsize': 28,
   'font.size': 28,
   'legend.fontsize': 26,
   'xtick.labelsize': 26,
   'ytick.labelsize': 26,
   "lines.linewidth": 2.5,
   'text.usetex': True,
   'figure.figsize': [12, 8],
    'font.family': 'sans-serif',
    'font.sans-serif': 'DejaVu Sans',
    'mathtext.fontset': 'dejavusans'
   }
plt.rcParams.update(params)
font = FontProperties().copy()

args.fontsize = params['legend.fontsize']
args.legend_fontsize = params['legend.fontsize']
args.labelsize =params['axes.labelsize']

import logging
mpl_logger = logging.getLogger('matplotlib')
mpl_logger.setLevel(logging.WARNING) 

In [63]:
# I/O - "configuration file" and parameters/options

# If you run the downsampled data, you will need to adjust the path to the configuration file and the results file
# args.baseloc abd args.dirloc, as well as the name of the files configs and alignment_counts_files.

args.baseloc = '/beegfs/pub/hbigs_course_2019/ribosome-profiling/riboSeqHBIGS19-analysis/config'
args.dirloc = '/beegfs/pub/hbigs_course_2019/ribosome-profiling/riboSeqHBIGS19-analysis/analysis'

# config file for the project to read in the sample name map
configs = {
    'hbigs': os.path.join(args.baseloc, "hbigs19.yaml")
}

alignment_counts_files = {
    'hbigs': os.path.join(args.dirloc, "HBIGS19.read-filtering-counts.csv.gz")
}

read_lengths = {
    'hbigs': os.path.join(args.dirloc, "HBIGS19.read-length-distributions-unique.csv.gz")
}

periodic_lengths = {
    'hbigs': os.path.join(args.dirloc, "HBIGS19.periodic-length-and-offsets.csv.gz")
}

counts = {
    'hbigs': os.path.join(args.dirloc, "HBIGS19.counts-per-frame.csv.gz")
}


data = 'hbigs'
title_str = 'Ribo-seq'

args.without_rrna = False
# args.without_rrna = True

# customise scale
ymax = 8.0e7
ystep = 1.0e7
ymax_without_rrna = 5.0e7
ystep_without_rrna = 1.0e7


In [36]:
# READ FILTERING COUNT

if args.without_rrna:
    args.ymax = ymax_without_rrna
    args.ystep = ystep_without_rrna
else:
    args.ymax = ymax
    args.ystep = ystep


args.alignment_counts_order = [
    'raw_data_count', 
    'without_adapters_count', 
    'without_rrna_count', 
    'genome_count', 
    'unique_count', 
    'length_count'
]

args.alignment_counts_names = [
    'Poor quality', 
    'Ribosomal', 
    'No alignment', 
    'Multimappers', 
    'Non-periodic', 
    'Usable'
]

args.without_rrna_order = [
    'without_rrna_count', 
    'genome_count', 
    'unique_count', 
    'length_count'
]

args.without_rrna_names = [
    "No alignment", 
    "Multimappers", 
    "Non-periodic", 
    "Usable"
]

if args.without_rrna:
    args.alignment_counts_order = args.without_rrna_order
    args.alignment_counts_names = args.without_rrna_names

args.alignment_counts = alignment_counts_files[data]

args.alignment_counts_order = args.alignment_counts_order[::-1]
args.alignment_counts_names = args.alignment_counts_names[::-1]


In [40]:
msg = "Reading counts"
logger.info(msg)

alignment_counts = pd.read_csv(args.alignment_counts)

config = yaml.load(open(configs[data]), Loader=yaml.FullLoader)
sample_name_map = ribo_utils.get_sample_name_map(config)

alignment_counts = alignment_counts.sort_values('note').reset_index()

names = alignment_counts['note']

alignment_diff_counts = mpl_utils.get_diff_counts(alignment_counts[args.alignment_counts_order])
df = pd.DataFrame(alignment_diff_counts)
df.columns = args.alignment_counts_names
df['name'] = names.reset_index(drop=True)

df['display_name'] = df['name'].apply(lambda x: sample_name_map[x])

In [39]:
fig, ax = plt.subplots()

pal = sns.palettes.color_palette(palette="Set3", n_colors=len(args.alignment_counts_names))

gap = .2
yticks = np.arange(0, args.ymax, args.ystep)

bars = mpl_utils.create_stacked_bar_graph(
    ax,
    alignment_diff_counts,
    colors=pal,
    x_tick_labels=df['display_name'],
    y_ticks=yticks,
    y_tick_labels=yticks,
    gap=gap,
    end_gaps=True,
    stack_labels=args.alignment_counts_names,
    y_title='Ribo-seq reads',
    log=False,
    font_size=args.fontsize,
    label_font_size=args.labelsize,
    edge_colors='1'
)

leg = ax.legend(bbox_to_anchor=(1, 0), 
                loc="lower left",
                bbox_transform=ax.transAxes, 
                columnspacing=0.5,
                fontsize=args.legend_fontsize,
                frameon=False,
                ncol=1)

if args.without_rrna:
    ax.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.0e'))
else:
    ax.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.0e'))
    
mpl_utils.set_label_fontsize(ax, args.fontsize)
mpl_utils.set_legend_title_fontsize(ax, args.fontsize)

ax.set_title(title_str)

In [54]:
# # FOOTPRINT LENGTH DISTRIBUTION

args.reads = read_lengths[data]
args.periodic = periodic_lengths[data]
args.out = read_lengths[data]

msg = "Reading counts"
logger.info(msg)

reads = pd.read_csv(args.reads, sep=',')
periodic = pd.read_csv(args.periodic, sep=',')

In [45]:
subset_samples = ['EGF', 'PBS'] # the two conditions
all_lengths_list = []
for sample in subset_samples:
    m_subset = periodic['condition'].str.contains('^'+sample)
    periodic_sub = periodic[m_subset]

    all_lengths = defaultdict(list) 
    all_names = periodic_sub['condition'].unique()
    for name in all_names:
        lengths = [str(l) for l in periodic_sub[periodic_sub['condition']==name].lengths.values]
        for l in lengths:
            all_lengths[l].append(reads[reads['condition']==name][lengths[lengths.index(l)]].values[0])

    all_lengths_df = pd.DataFrame.from_dict(all_lengths, orient='index').T.unstack().reset_index() 
    all_lengths_df['Model'] = sample
    all_lengths_list.append(all_lengths_df)
    
all_lengths_df = pd.concat(all_lengths_list)
all_lengths_df = all_lengths_df[~all_lengths_df[0].isna()].copy()
all_lengths_df.rename(columns={'level_0':'Periodic footprint length (nt)', 0:'Ribo-seq reads'}, inplace=True)


In [50]:
fig, ax = plt.subplots()

pal = sns.palettes.color_palette(palette="Set3", n_colors=6)
pal = [pal[4], pal[3]]

flatui = ["#3498db", "#e74c3c"] # "#95a5a6" "##9b59b6"
hue_palette = sns.color_palette(flatui)

# sns.catplot(x="level_0", y=0, kind="swarm", data=df)
g = sns.swarmplot(x="Periodic footprint length (nt)", y='Ribo-seq reads', data=all_lengths_df, color=pal[0], size=8, ax=ax,
                 edgecolor=pal[0], hue='Model', hue_order=subset_samples, 
                  palette=hue_palette)

leg = ax.legend(loc="upper right",
                columnspacing=0.5,
                fontsize=params['legend.fontsize'],
                frameon=False,
                ncol=1)

sns.despine()

In [56]:
# # COUNTS PER FRAME

args.counts = counts[data]

msg = "Reading counts"
logger.info(msg)

frame_counts = pd.read_csv(args.counts)
    
frame_counts = frame_counts.sort_values('condition').reset_index()

# From raw value to percentage

totals = [i+j+k for i,j,k in zip(frame_counts['frame_count'], frame_counts['frame+1_count'], frame_counts['frame+2_count'])]
frame1 = [i / j * 100 for i,j in zip(frame_counts['frame_count'], totals)]
frame2 = [i / j * 100 for i,j in zip(frame_counts['frame+1_count'], totals)]
frame3 = [i / j * 100 for i,j in zip(frame_counts['frame+2_count'], totals)]

In [57]:
fig, ax = plt.subplots()

pal = sns.palettes.color_palette(palette="Set3", n_colors=6)

names = frame_counts['condition']
r = range(len(names))

barWidth = .95
ax.bar(r, frame1, color=pal[0], edgecolor='white', width=barWidth, label='Reading frame')
ax.bar(r, frame2, bottom=frame1, color=pal[1], edgecolor='white', width=barWidth, label='Reading frame +1')
ax.bar(r, frame3, bottom=[i+j for i,j in zip(frame1,frame2)], color=pal[2], edgecolor='white', width=barWidth, label='Reading frame +2')
plt.xticks(r, names, rotation='vertical')
# ax.set_ylabel('Periodic,\nP-site shifted reads (\%)')
ax.set_ylabel('P-sites (\%)')
ax.legend(loc='upper right',
             bbox_to_anchor=(1.45, 1.),
    ncol=1,
    frameon=False,
    framealpha=0.9
)
ax.set_title(title_str)
sns.despine(left=True)

### Preprocessing analysis
***

In addition to the above, the `create-rpbp-preprocessing-report` script can be used to generate a latex document and several plots that summarize the preprocessing and ORF profile construction (among which you will recognise the first figure above, generated using the first 6 cells). During the report creation step, a file *read-filtering-counts.csv.gz* will be generated, which you can use to create the *read filtering count figures* above.

We have ran this script on the data, and we will examine the report, located under */beegfs/pub/hbigs_course_2019/ribosome-profiling/riboSeqHBIGS19-analysis/riboseq-results/HBIGS19preprocessing-report/preprocessing-report.pdf*.


&#9655;  **Self-guided learning**  You can try running the preprocessing analysis yourself on the downsampled data, using the previous notebook (making sure you are in the course directory under *riboSeqHBIGS19-downsampled-analysis/config*). To do that, copy the cell starting with the `%%sbatch` line, remove all lines from 9, starting with `run-all-rpbp-instances`, and replace them with the following lines. You need to adjust the second line and replace *course01* with your username.


In [ ]:
create-rpbp-preprocessing-report hbigs19-downsampled.yaml \
    /home/course01/hbigs_course_2019/ribosome-profiling/riboSeqHBIGS19-downsampled-analysis/riboseq-results/report \
    --num-cpus 6 \
    --mem 120G \
    --logging-level INFO \
    --log-file report-downsampled.log
    

## 1.3.3 Prediction analysis: understand the output of rpbp

In this section, we will briefly explore the structure of the output of `rpbp`, select a number of examples, and visualise the ORF predictions in a descriptive way. 

&#9655; **Beyond this introduction** The ORF predictions can be used as starting point for an indepth analysis of a dataset/experiment, by performing *e.g.* conservation, sequence/motif analyses, *etc.* or be used in combination with RNA-seq to perform translational efficiency analysis (see Part 2).


**Note** There is a command that can be used to generate a report (similar to the preprocesing analysis), called `create-rpbp-predictions-report` (see [Predictions analysis](https://rp-bp.readthedocs.io/en/latest/analysis-scripts.html#predictions-analysis)). We are currently working on a "next-generation" solution to the pdf report/figures!

***
<font color=red>**Note** The cells below contain "code", so we will need to run them one after the other.</font> 



In [68]:
import pbio.utils.bed_utils as bed_utils

orfs = {
    'hbigs': os.path.join(args.dirloc, "HBIGS19.filtered.predicted-orfs.bed.gz")
}
args.orfs = orfs[data]
predicted_orfs = bed_utils.read_bed(args.orfs)


In [89]:
# The results are displayed as an extended BED file.
# See https://www.ensembl.org/info/website/upload/bed.html (BED File Format)

# We run this cell, with the name of the results table, called a dataframe, to see the output.
# We will explain the different fields...

predicted_orfs


In the paper, they show how EGF stimulation results in distinct "waves" of protein synthesis. Proteins whose synthesis was statistically significantly altered in at least one time point were subjected to hierarchical clustering.

We pick 2 genes from cluster 1 (characterized by increased protein synthesis as early as 30 min following EGF stimulation): EGR1 (refered to as a canonical immediate early gene or IEG, *i.e* a gene that is rapidly up-regulated within the first hour following stimulus) and ATF3 (a "classical" delayed early gene; in contrast to established literature, this one appears to be rapidly down-regulated after 15 min following EGF stimulation, then up-regulated after 30 min).

In [147]:
mEGR1 = predicted_orfs['id'].str.contains('ENST00000239938') # one EGR1 transcript
predicted_orfs[mEGR1]


In [146]:
mATF3 = predicted_orfs['id'].str.contains('ENST00000492118') # one ATF3 transcript
predicted_orfs[mATF3]

# What do you notice?

What do you notice from the output above?

<br>
<br>
<div>
<img src="img/Human_ATF3.png" width="900" align="center"/>
</div>
<br>
<br>

ATF3 is a gene encoding an activation transcription factor. This gene is induced by a variety of signals, including cellular stress response. 

Another member of this family is ATF4. The translation of ATF4 is known to be dependent on uORFs. In particular, during normal conditions, uORFs repress translation of the main ATF4 CDS, while under stress conditions ATF4 is translated.

What do you notice from the output below?

In [145]:
mATF4 = predicted_orfs['id'].str.contains('ENST00000404241') # one ATF4 transcript
predicted_orfs[mATF4]


We will now use all predictions from the `rpbp` pipeline, and look at the distribution of ORFs (*i.e.* translatable ORFs which were predicted with a certain confidence to actually be translated, using the Ribo-seq data).


In [162]:
# get the ORF counts per strand

orf_type_counts = predicted_orfs.groupby(['orf_type', 'strand']).size()
orf_type_counts = orf_type_counts.reset_index(name="count")
orf_type_counts['display_name'] = orf_type_counts['orf_type'].map(ribo_utils.orf_type_display_name_map)

# orf_type_counts = orf_type_counts[orf_type_counts['orf_type']!='canonical']

In [161]:
orf_type_counts

In [165]:
color = sns.palettes.color_palette("Set3", n_colors=3)

fig, ax = plt.subplots(figsize=(9,5))
sns.barplot(
    x="display_name",
    y="count",
    hue="strand",
    data=orf_type_counts,
    ax=ax,
    zorder=-1,
    palette='Set3',
    log=True
)

sns.despine()

ax.legend(
    loc='upper right', 
    bbox_to_anchor=(1.3, 1.3), 
    fontsize=params['legend.fontsize'],
    frameon=False,
    title="Strand"
)
mpl_utils.set_legend_title_fontsize(ax, args.fontsize)

ax.set_ylabel("Number of ORFs", fontsize=args.fontsize)
ax.set_xlabel("", fontsize=0)

# rotate the ORF type names
mpl_utils.set_ticklabels_fontsize(ax, args.fontsize)
mpl_utils.set_ticklabel_rotation(ax, axis='x', rotation=90)

# place the ORF type names in the middle of the bar
for ticklabel in ax.xaxis.get_ticklabels():    
    p = ticklabel.get_position()
    ticklabel.set_position((p[0], 0.1))
    ticklabel.set_verticalalignment('bottom')

ax.set_title(title_str, fontsize=args.fontsize)



***

MIT License (code and scripts)

Copyright (c) 2019 Etienne Boileau